# Iteraatio 3 - kohteiden ominaisuuksien valintaa gini-indeksillä

In [2]:
import numpy as np
import pandas as pd

WORKING_DIRECTORY = '/mnt/c/git/masters-thesis-code/jupyter/code/'

## Ladataan yritysdata

In [3]:
SELECTED_COMPANY_FEATURES = ['company_form_code', 'location_municipality_code', 
                             'location_region_code', 'company_status_code', 'industry_code', 'turnover', 
                             'net_profit', 'personnel_average', 'performer_ranking_points', 'risk_rating_class']

In [4]:
COMPANIES_DF = pd.read_pickle(WORKING_DIRECTORY + "data/pandas_pickles/prod_data_proto2.pkl")

In [5]:
ITEM_IDS = list(COMPANIES_DF['business_id'].unique())

## Ladataa vuorovaikutusdata

In [43]:
interactions_tmp = pd \
    .read_csv(WORKING_DIRECTORY + 'data/interactions_2021_08_19.csv',
             delimiter='\t',
             dtype={
                 'group_id': 'string',
                 'business_id': 'string',
                 'owner': 'string'
             })

# otetaan pois 1 kokoiset ryhmät
group_sizes = interactions_tmp['group_id'].value_counts()
group_sizes_df = pd.DataFrame({'group_id': group_sizes.index, 'group_size': group_sizes.values})

INTERACTIONS_WITH_GROUP_SIZES_DF = interactions_tmp.merge(group_sizes_df, on='group_id')             
interactions_tmp = INTERACTIONS_WITH_GROUP_SIZES_DF[INTERACTIONS_WITH_GROUP_SIZES_DF.group_size >= 2]
#interactions_tmp = INTERACTIONS_WITH_GROUP_SIZES_DF[INTERACTIONS_WITH_GROUP_SIZES_DF.group_size <= 3000]
interactions_tmp.sort_values('group_size')

,group_id,business_id,owner,group_size
155696,3e9dd356-2b21-45ae-9ee4-7cd6cc122fe1,07577937,5e87095492119e00066e7158,2
106198,31503959-943a-4081-abcc-dc80e5cb0402,15093748,5db034c64320cd0006d2b788,2
313746,cab22fae-db47-46b6-b902-3d9a1b1051f6,01163004,5e4534bc7bf061000697e940,2
313747,cab22fae-db47-46b6-b902-3d9a1b1051f6,10410900,5e4534bc7bf061000697e940,2
545392,0967d6ed-88b7-4023-a720-f09f7051f24d,17944788,5efdbc656488210007bc27f6,2
...,...,...,...,...
8042,a5c6ce2e-22ab-4871-bd72-e5da294b33cc,16029641,5e1489f3c2f568000654ecbb,3999
8043,a5c6ce2e-22ab-4871-bd72-e5da294b33cc,16030167,5e1489f3c2f568000654ecbb,3999
8044,a5c6ce2e-22ab-4871-bd72-e5da294b33cc,16030415,5e1489f3c2f568000654ecbb,3999
8031,a5c6ce2e-22ab-4871-bd72-e5da294b33cc,16001948,5e1489f3c2f568000654ecbb,3999


In [44]:
# lisätään konserniyrityksille interaktiot
concern_interactions = interactions_tmp.copy()
concern_interactions['business_id'] = 'K-' + concern_interactions['business_id'].astype(str)
concern_interactions = concern_interactions[concern_interactions.business_id.isin(ITEM_IDS)]
concern_interactions

,group_id,business_id,owner,group_size
5,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-01681709,60646431ae18cb00063ed63f,1862
6,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-15055514,60646431ae18cb00063ed63f,1862
7,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-01876143,60646431ae18cb00063ed63f,1862
9,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-05363070,60646431ae18cb00063ed63f,1862
10,c2626398-faac-4ff3-b02d-cdc64b50cdaa,K-01387534,60646431ae18cb00063ed63f,1862
...,...,...,...,...
548074,8b0915ff-a0cb-4520-9160-8d783a6bf308,K-02106319,6110c56241e21e000857ca77,131
548110,8b0915ff-a0cb-4520-9160-8d783a6bf308,K-20333371,6110c56241e21e000857ca77,131
548137,8b0915ff-a0cb-4520-9160-8d783a6bf308,K-07027249,6110c56241e21e000857ca77,131
548162,8b0915ff-a0cb-4520-9160-8d783a6bf308,K-02011774,6110c56241e21e000857ca77,131


In [45]:
# yhdistetään konserni-interaktiot tavallisiin ja poistetaan sellaiset interaktiot, joille ei löydy y-tunnusta
INTERACTIONS_DF = pd.concat([interactions_tmp, concern_interactions])
INTERACTIONS_DF = INTERACTIONS_DF[INTERACTIONS_DF.business_id.isin(ITEM_IDS)]
INTERACTIONS_DF[INTERACTIONS_DF['business_id'] == 'K-02011774']

,group_id,business_id,owner,group_size
18663,6c894b42-cbc6-4d18-8cf1-39ee91d2bf53,K-02011774,6058a712ae18cb00063ed639,2175
49204,66aea578-682d-45de-a200-77fa79a8c5e7,K-02011774,5db92c0ebc3e9100062ac0b0,3692
217786,608ac4ff-ab75-425b-a4f1-74d5defd43a3,K-02011774,5fbe6408f464de0006491d9e,1541
231830,4d2c2290-72be-40fe-8506-42718d2aac25,K-02011774,5f5613da9769490006b0ebb3,3685
254761,58f4cb31-0d40-42b2-9268-10d6dc64f1a0,K-02011774,5f5613da9769490006b0ebb3,2595
298418,2177a613-3b16-483e-9c9b-dc20bd4225f6,K-02011774,608863795602580007e70ddf,3925
354226,62c0017d-ff32-4584-8c85-562e9a1e8329,K-02011774,5fab9e2dc07a1900066bab26,3949
412123,3a44f374-56ee-4500-8829-c4b942b7afec,K-02011774,6033ae80484e8a0006fe437a,3966
439385,725e7cd0-7031-4988-b4cb-69672c611514,K-02011774,5db83a1cbc3e9100062ac0ab,1846
451491,acfdacd6-0de0-4568-a819-c3c2e15ef221,K-02011774,6033ae80484e8a0006fe437a,3809


## Poimitaan käytössä olevat ominaisuudet

In [46]:
features_tmp = [COMPANIES_DF[feature].unique() for feature in SELECTED_COMPANY_FEATURES]
FEATURES = [item for sublist in features_tmp for item in sublist]
len(FEATURES)
FEATURES

['company_form_code+CO_26',
 'company_form_code+CO_16',
 'company_form_code+CO_53',
 'company_form_code+CO_2',
 'company_form_code+CO_10',
 'company_form_code+CO_13',
 'company_form_code+CO_19',
 'company_form_code+CO_5',
 'company_form_code+CO_14',
 'company_form_code+CO_6',
 'company_form_code+CO_63',
 'company_form_code+CO_51',
 'company_form_code+CO_56',
 'company_form_code+CO_18',
 'company_form_code+CO_60',
 'company_form_code+CO_17',
 'company_form_code+CO_0',
 'company_form_code+CO_54',
 'company_form_code+CO_52',
 'company_form_code+CO_50',
 'company_form_code+CO_57',
 'company_form_code+CO_31',
 'company_form_code+CO_29',
 'company_form_code+CO_41',
 'company_form_code+CO_71',
 'company_form_code+CO_39',
 'company_form_code+CO_36',
 'company_form_code+CO_90',
 'company_form_code+CO_48',
 'company_form_code+CO_44',
 'company_form_code+CO_59',
 'company_form_code+CO_55',
 'company_form_code+CO_42',
 'company_form_code+CO_40',
 'company_form_code+CO_15',
 'company_form_code+CO_7

## Lasketaan gini-indeksit

In [47]:
def calculate_gini_for_word(word):
    col_name = word.split('+')[0]
    matches_df = COMPANIES_DF[COMPANIES_DF[col_name] == word]
    
    matched_docs_total = matches_df.shape[0]
    
    match_bids = list(matches_df['business_id'].unique())
    
    matching_interactions_df = INTERACTIONS_DF[INTERACTIONS_DF['business_id'].isin(match_bids)]
    
    interacted_docs_count = matching_interactions_df['business_id'].unique().shape[0]
    non_interacted_docs_count = matched_docs_total - interacted_docs_count
    
    gini_index = 1 - ((interacted_docs_count / matched_docs_total) ** 2 + \
                    (non_interacted_docs_count / matched_docs_total) ** 2)
    
        
    return (word, gini_index, interacted_docs_count, matched_docs_total)

In [48]:
print(calculate_gini_for_word('location_municipality_code+NaN'))

('location_municipality_code+NaN', 0.08463206422198355, 39337, 888441)


In [49]:
GINI_SCORES = []

for word in FEATURES:
    gini = calculate_gini_for_word(word)
    GINI_SCORES.append(gini)

In [50]:
GINI_DF = pd.DataFrame(GINI_SCORES, columns=['feature', 'gini_score', 'interacted_count', 'total_count'])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(GINI_DF.sort_values('gini_score'))

                            feature  gini_score  interacted_count  total_count
487                industry_code+06    0.000000                 0            2
36          company_form_code+CO_58    0.000000                 0           24
48          company_form_code+CO_21    0.000000                 0            3
51          company_form_code+CO_28    0.000000                 0            5
52           company_form_code+CO_9    0.000000                 1            1
53           company_form_code+CO_1    0.000000                 0            5
54          company_form_code+CO_35    0.000000                 0            9
27          company_form_code+CO_90    0.000000                 0           54
56           company_form_code+CO_3    0.000000                 0            2
58          company_form_code+CO_20    0.000000                13           13
16           company_form_code+CO_0    0.000000                 0         2915
60           company_form_code+CO_7    0.000000     